In [ ]:
%matplotlib inline
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import models
import numpy as np
import math
import random
import pandas as pd
import cv2
import time
np.random.seed(1337)

In [ ]:
TrainImage = torch.load('data/TrainImg.pth')
TrainLabel = torch.load('data/TrainLabel.pth')

In [ ]:
TrainImages = torch.load('data/TrainImgs.pth')
TrainLabels = torch.load('data/TrainLabels.pth')

In [ ]:
def sufflefinal(it,il,N,m):
    for loop in range(N):
        irand1 = random.randint(0, m-1)
        irand2 = random.randint(0, m-1)
        
        ivar = it.index_select(0, torch.LongTensor([irand1]))
        it[irand1] = it.index_select(0, torch.LongTensor([irand2]))
        it[irand2] = ivar
    
        ivar = il[irand1]
        il[irand1] = il[irand2]
        il[irand2] = ivar
        
    return it,il

In [ ]:
net = models.densenet201(pretrained=True)

new_classifier = nn.Sequential(*list(net.classifier.children())[:-1])
new_classifier.add_module('classifier',nn.Linear(1920,10))
net.classifier = new_classifier

net = net.cuda()
net = torch.nn.DataParallel(net, device_ids=[0, 2])

In [ ]:
TrainImage = ((TrainImage.float())/255.0)
TrainLabel = TrainLabel.long()

In [ ]:
TrainImages = ((TrainImages.float())/255.0)
TrainLabels = TrainLabels.long()

In [ ]:
criterion = nn.CrossEntropyLoss(size_average=False)
optimizer = optim.Adam(net.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,100,150,200], gamma=0.1)

In [ ]:
iterations = 250
BatchSize = 250
for epoch in range(iterations):
    epochStart = time.time()
    runningLoss = 0.0
    TrainImages, TrainLabels = sufflefinal(TrainImages, TrainLabels,1000,275*80)
    TrainImage, TrainLabel = sufflefinal(TrainImage, TrainLabel,1000,275*10)
    scheduler.step()
    for i in range(11):
        inputs = torch.index_select(TrainImage,0,torch.linspace(i*BatchSize,(i+1)*BatchSize - 1,steps=BatchSize).long())
        labels = torch.index_select(TrainLabel,0,torch.linspace(i*BatchSize,(i+1)*BatchSize - 1,steps=BatchSize).long())
        optimizer.zero_grad()
        outputs = net(Variable(inputs).cuda())
        loss = criterion(outputs, Variable(labels).cuda())
        loss.backward()
        runningLoss+=loss.data[0]
        mul=0.7
        for myparam in optimizer.param_groups:
            for mygrad in myparam['params']:
                mygrad.grad.data = mul*(mygrad.grad.data)
        optimizer.step()
        
        for j in range(8):
            inputs = torch.index_select(TrainImages,0,torch.linspace(i*BatchSize*8+j*BatchSize,i*BatchSize*8+(j+1)*BatchSize-1,steps=BatchSize).long())
            labels = torch.index_select(TrainLabels,0,torch.linspace(i*BatchSize*8+j*BatchSize,i*BatchSize*8+(j+1)*BatchSize-1,steps=BatchSize).long())
            optimizer.zero_grad()
            outputs = net(Variable(inputs).cuda())
            loss = criterion(outputs, Variable(labels).cuda())
            loss.backward()
            runningLoss+=loss.data[0]
            mul=0.0375
            for myparam in optimizer.param_groups:
                for mygrad in myparam['params']:
                    mygrad.grad.data = mul*(mygrad.grad.data)
            optimizer.step()

    epochEnd = time.time()-epochStart  
    print('At Iteration: %d / %d ;Tr Loss: %f ;Time: {%f}m {%f}s'%(epoch + 1,iterations,runningLoss/24750,epochEnd//60,epochEnd%60))
print('Finished Training')

In [ ]:
listing1 = ['Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

Testpath = 'test/'
listing2 = os.listdir(Testpath)

In [ ]:
outImgname = []
outlabel = []
Tsdata = torch.FloatTensor(1,3,224,224)
for i in range(len(listing2)):
    image = cv2.imread(Testpath+listing2[i], flags=cv2.IMREAD_COLOR)
    w,h,c = image.shape
    image = image[144:368,144:368].transpose(2,0,1)
    Tsdata[0] = torch.from_numpy(image)
    
    Tsdata = Tsdata/255.0
    net.eval()
    out = net(Variable(Tsdata).cuda())
    _, predicted = torch.max(out.data.cpu(), 1)
    outlabel.append(listing1[predicted[0]])
    outImgname.append(listing2[i])

In [ ]:
submission = pd.DataFrame({'fname': np.array(outImgname), 'camera': np.array(outlabel)})
submission.to_csv('ieee_sub.csv', index=False)